In [1]:
# TIME STAMP
import os
import time

def watermark(path):
    print("Created: %s" % time.ctime(os.path.getctime(path)))
    print("Last modified: %s" % time.ctime(os.path.getmtime(path)))
    print("Author: Mattithyahu")

# Always remember to use unicode escape for backslashes if present 
watermark("P1 Euro 2020 Correlation.ipynb")

Created: Sun Oct 24 01:54:01 2021
Last modified: Thu Oct 28 16:13:16 2021
Author: Mattithyahu


In [2]:
# TABLE 1 

import pandas as pd

url = "https://fbref.com/en/comps/676/UEFA-Euro-Stats#all_results1027811"
dfs = pd.read_html(url)

# Becasue there is only one table on the page with len == 28
for df in dfs:
    if len(df) >= 28:
        the_one = df
        break

# assigning the_one a better name
df = the_one 


# Cleaning Data

# Deleting notes column
# df = df.drop('Notes', 1)
# where 1 is the axis number (0 for rows and 1 for columns.)

# To delete the column without having to reassign df you can use inplace=True
# Column to be deleted defined as 'Notes'. 
df.drop('Notes', axis=1, inplace=True)

# After column with all Null values taken out using dropna takes out any rows where null values exist 
# inplace=True replaces the df with the new UPDATED df 
# df['Value'].replace({'%':''}, regex=True).astype('float')
# Note, that I had to use the regex=True parameter for this to work since the % is a part of the string and not the full string value.
# Cleaning %s from data
# df = df.replace({'%': ''}, regex=True)
# df[['col1', 'col2', 'col3']] = df[['col1', 'col2', 'col3']].apply(pd.to_numeric)
# df.info()
# df.dropna(inplace=True)

# When using expand=True, the split elements will expand out into separate columns. If NaN is present, it is propagated throughout the columns during the split.
df[['col1', 'col2']] = df['Squad'].str.split(n=1, expand=True)


# Deleting squad column as it doesnt provide any useful information AND renaming split column to Team
df.drop('Squad', axis=1, inplace=True)
df.drop('col1', axis=1, inplace=True)
df = df.rename(columns={"col2": "Team"})

# saving scraped df to csv 
df.to_csv('C:/Users/matti/OneDrive/MyProjects/Data/P1 Euro 2020 Correlation/Team playing stats.csv', index=False)

# printing df
df

Total tables: 28


In [4]:
# TABLE 2

# Reading data from specific table items
# KEY: Avoid reading data that is not within the table that has the same class name as a column 
import requests
from bs4 import BeautifulSoup
import pandas as pd

# The term refers to any piece of software that facilitates end-user interaction with web content. A user agent (UA) string is a text that the client computer software 
# sends through a request.
# The user agent string helps the destination server identify which browser, type of device, and operating system is being used. 
# For example, the string tells the server you are using Chrome browser and Windows 10 on your computer. The server can then use this information to adjust the response for 
# the type of device, OS, and browser.
# When you are web scraping, sometimes you will find that the webserver blocks certain user agents. This is mostly because it identifies the origin as a bot and certain 
# websites don’t allow bot crawlers or scrapers.
# Find Your User-Agent: https://httpbin.org/get
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.54 Safari/537.36"}

# url of the page defined
url2 = "https://www.transfermarkt.co.uk/euro-2020/teilnehmer/pokalwettbewerb/EM20"
# Requests allows a user to ake a request to a web page and The get() method sends a GET request to the specified url. headers deines your User Agent
pageTree = requests.get(url2, headers=headers)
# content returns the content of the response, in bytes. This module defines a class HTMLParser which serves as the basis for parsing text files formatted in HTML
pageSoup = BeautifulSoup(pageTree.content, 'html.parser')

# Creating list using pageSoup to find tanle in sight calling the class and id 
data = []
table = pageSoup.find('table', attrs={'class':'items'})
#The <tbody> tag is used to group the body content in an HTML table.
table_body = table.find('tbody')

# Assigning table rows to row. For a row in rows find the 'td'(column)
# Getting rid of empty values
rows = table_body.find_all('tr')
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    data.append([ele for ele in cols if ele]) 

df1 = pd.DataFrame(data)

# Cleaning Data
# Renaming dataframe with correct column headers 
df1 = df1.rename(columns={0: "Team", 1: "avg_age", 5: "market_value", 6: "avg_market_value"})

# dropping columns that arent needed
df1.drop(2, axis=1, inplace=True)
df1.drop(3, axis=1, inplace=True)
df1.drop(4, axis=1, inplace=True)

# # saving scraped df to csv 
df1.to_csv('C:/Users/matti/OneDrive/MyProjects/Data/P1 Euro 2020 Correlation/Team Valuation.csv', index=False)
df1


,Team,avg_age,market_value,avg_market_value
0,England,24,£1.03bn,£42.75m
1,France,23,£877.32m,£38.14m
2,Germany,23,£808.65m,£35.16m
3,Spain,23,£635.40m,£27.63m
4,Italy,22,£629.10m,£28.60m
5,Netherlands,27,£617.85m,£22.88m
6,Portugal,23,£593.55m,£25.81m
7,Belgium,20,£448.20m,£22.41m
8,Croatia,24,£292.50m,£12.19m
9,Denmark,23,£266.18m,£11.57m


In [5]:
# TABLE 3

import requests 
from bs4 import BeautifulSoup

url3 = "https://www.90min.com/posts/euro-2020-the-final-team-rankings"

headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.54 Safari/537.36"}

page2Tree = requests.get(url3, headers=headers)
page2Soup = BeautifulSoup(page2Tree.content, 'html.parser')

# data = []
test = page2Soup.findAll('span', attrs={'class':'tagStyle_q7y9ey-o_O-numberStyle_l6szse-o_O-tagStyle_1igopqi'})

testb = page2Soup.findAll('h2', attrs={'class':'tagStyle_1igopqi-o_O-style_48hmcm-o_O-titleStyle_u23ta5'})

# test[0].text 
# Only h2 needed as it outputs rank too 
testb[0].text

# Itersting to create dataframe using a for loop
rows = [[testb[i].text] for i in range(24)]
df2 = pd.DataFrame(rows, columns=["A"])

# Splitting into different columns but the first space (n=1)
df2[['Rank', 'Team']] = df2['A'].str.split(n=1, expand=True)

# Splitting new rank column to seperate number from dot
a = df2['Rank'].str.partition('.')
  
# df2
# a

# Adding column from dataframe a to df2 with name 'ACC rank'
df2['ACC rank'] = a[0].values

# Dropping columns A and Rank as they have been cleaned and recreated and are no longer needed
df2.drop('A', axis=1, inplace=True)
df2.drop('Rank', axis=1, inplace=True)

# renaming column name
df2 = df2.rename(columns={"ACC rank": "Rank"})
df2.sort_values(['Rank'], inplace=True)


df2.to_csv('C:/Users/matti/OneDrive/MyProjects/Data/P1 Euro 2020 Correlation/Team rank.csv', index=False)


df2


,Team,Rank
23,Italy,1
14,Portugal,10
13,Netherlands,11
12,Germany,12
11,Croatia,13
10,Austria,14
9,Sweden,15
8,Wales,16
7,Hungary,17
6,Finland,18


In [6]:
# TABLE 4  JOIN 

# If we want to join using the key columns, we need to set key to be the index in both df and other. The joined DataFrame will have key as its index.
# Important to inner join them so the index is kept 
JOIN = df1.merge(df2, how='inner', on='Team')

# JOIN = df2.set_index('Team').join(df1.set_index('Team'))
JOIN.sort_values(['Rank'], inplace=True)
# JOIN= JOIN.sort_values(by='Rank')
# JOIN.drop('Country', axis=0, inplace=True)

# Changing bn to million for England using loc
JOIN.loc[JOIN['Team'] == 'England', 'market_value'] = '£1030.00m'

# Because we know for a fact the £ and m will be the first and last character we can use 
JOIN['market_value'] = JOIN['market_value'].str[1:-1]
JOIN['avg_market_value'] = JOIN['avg_market_value'].str[1:-1]

# Converting cleaned columns to numeric so calculations can be performed on it
JOIN["Rank"] = pd.to_numeric(JOIN["Rank"])
JOIN["avg_age"] = pd.to_numeric(JOIN["avg_age"])
JOIN["market_value"] = pd.to_numeric(JOIN["market_value"])
JOIN["avg_market_value"] = pd.to_numeric(JOIN["avg_market_value"])

# Column can now be sorted by Rank as its now numeric 
# JOIN.sort_values(['Rank'], inplace=True, ascending=False)

# Used to identify quartiles in the market_value 
JOIN.market_value.quantile([0,0.25,0.5,0.75,1])

# Creating market value field to group team market values in 4 brackets 
JOIN['market_value_Quartile'] = (
    pd.cut(
        JOIN['market_value'], bins=[33.1700, 122.1525, 226.5750, 599.6250, 1030.00],
        labels=['Q1', 'Q2', 'Q3', 'Q4'], include_lowest=True)
)

JOIN.sort_values(['market_value_Quartile'], inplace=True, ascending=False)


JOIN.to_csv('C:/Users/matti/OneDrive/MyProjects/Data/P1 Euro 2020 Correlation/Value and rank JOIN.csv', index=False)
JOIN.to_csv('Value and rank JOIN.csv', index=False)
JOIN


,Team,avg_age,market_value,avg_market_value,Rank,market_value_Quartile
4,Italy,22,629.10,28.60,1,Q4
0,England,24,1030.00,42.75,2,Q4
3,Spain,23,635.40,27.63,4,Q4
1,France,23,877.32,38.14,9,Q4
5,Netherlands,27,617.85,22.88,11,Q4
2,Germany,23,808.65,35.16,12,Q4
8,Croatia,24,292.50,12.19,13,Q3
10,Turkey,24,264.24,11.01,24,Q3
7,Belgium,20,448.20,22.41,5,Q3
6,Portugal,23,593.55,25.81,10,Q3


In [7]:
import plotly.express as px


fig = px.scatter(JOIN, x="Rank", y="market_value",trendline="ols", hover_name= "Team" ,size='market_value', title="Rank to Team Value ",labels={
                     "Rank": "Euro 2020 Ranking",
                     "market_value": "Team Market value (millions) (£)",
                     "market_value_Quartile": "Value Quartile"
                 },)

fig2 = px.scatter(JOIN, x="Rank", y="avg_market_value",trendline="ols", hover_name= "Team" ,size='market_value', title="Rank to AVG Team Value ",labels={
                     "Rank": "Euro 2020 Ranking",
                     "market_value": "Team Market value (millions) (£)",
                     "market_value_Quartile": "Value Quartile"
                 },)

fig3 = px.scatter(JOIN, x="Rank", y="avg_age",trendline="ols", hover_name= "Team" ,size='market_value', title="Rank to AVG Team Age ",labels={
                     "Rank": "Euro 2020 Ranking",
                     "market_value": "Team Market value (millions) (£)",
                     "market_value_Quartile": "Value Quartile"
                 },)
                 

# fig.update_layout(color="green" if "market_value_Quartile" == 'Q1' else "red")
# color="market_value_Quartile" 

# fig.update_traces(marker=dict( color='red' ))

fig.update_layout(
    font=dict(
        family="Courier New, monospace",
        size=14,
        color="black"
    ),
    title={
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'}
)

fig2.update_layout(
    font=dict(
        family="Courier New, monospace",
        size=14,
        color="Black"
    ),
    title={
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'}
)

fig3.update_layout(
    font=dict(
        family="Courier New, monospace",
        size=14,
        color="Black"
    ),
    title={
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'}
)



# To reverse x axis
fig.update_xaxes(autorange="reversed")
fig2.update_xaxes(autorange="reversed")
fig3.update_xaxes(autorange="reversed")

fig.show()
fig2.show()
fig3.show()

fig.write_image("Rank to Team Value.png")
fig2.write_image("Rank to AVG Team Value.png")
fig3.write_image("Rank to AVG Team Age.png")